In [14]:
print("all_ok")

all_ok


In [17]:
from langchain_groq import ChatGroq

In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
model=ChatGroq(model="qwen/qwen3-32b")
#model=ChatGroq(model="deepseek-r1-distill-llama-70b")

print(model.invoke("What is the capital of France?"))

content='<think>\nOkay, the user is asking for the capital of France. Let me think... I remember from geography that France is a country in Europe. The capital... I believe it\'s Paris. But wait, am I sure? Let me double-check. Sometimes countries have cities with similar names, but Paris is pretty unique. I think that\'s right. Yeah, Paris is the capital and also a major city in France. It\'s known as the City of Light, right? And it\'s famous for the Eiffel Tower and the Louvre. I don\'t think there\'s any confusion with other cities. Yeah, I\'m pretty confident the answer is Paris. No need to complicate it further.\n</think>\n\nThe capital of France is **Paris**. Known as the "City of Light," Paris is renowned for its cultural landmarks, such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is a major global hub for art, fashion, and gastronomy. \n\nLet me know if you\'d like additional details! 🇫🇷' additional_kwargs={} response_metadata={'token_usage': {'complet

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [20]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [15]:
embedding_model.embed_query("What is the capital of France?")

[0.04257791489362717,
 -0.047810737043619156,
 -0.02702580951154232,
 -0.035097863525152206,
 0.05324113741517067,
 0.0018493696115911007,
 0.004823467694222927,
 -0.022051338106393814,
 0.0009697225177660584,
 0.07324519753456116,
 -0.014812891371548176,
 0.003644853364676237,
 -0.00034491211408749223,
 0.028128888458013535,
 0.025020018219947815,
 -0.04156218096613884,
 0.005471833515912294,
 0.02652869001030922,
 0.043672770261764526,
 -0.014782802201807499,
 0.013127882033586502,
 0.007567551918327808,
 -0.03469207510352135,
 0.023462051525712013,
 0.020962651818990707,
 -0.05559537559747696,
 0.00859882216900587,
 -0.04824773594737053,
 -0.012368501164019108,
 -0.001532181748189032,
 -0.07255345582962036,
 0.04269229248166084,
 0.00527808116748929,
 -0.015593086369335651,
 0.02645784802734852,
 -0.0531519278883934,
 -0.0004922056687064469,
 0.016876710578799248,
 -0.00814562477171421,
 0.04225021228194237,
 -0.015036126598715782,
 -0.003916633781045675,
 -0.04687097668647766,
 0.0

In [ ]:
## 1. Data injestion into vector DB

In [1]:
from langchain.document_loaders import PyPDFLoader

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [4]:
import os
file_path=os.path.join(os.getcwd(), "data", "sample.pdf" )

In [7]:
loader=PyPDFLoader(file_path)
documents=loader.load()

In [ ]:
#We can take the entire document(77 pages) or split it into smaller chunks.
#small chunks are better for vector DBs and helps in preserving the context.



In [11]:
# this is a experimental value, there is no one size fits all. Hyperparameter tuning is required.
# study various chunking state-of-the-art techniques.
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500, 
    chunk_overlap=200, 
    length_function=len
)



In [12]:
docs=text_splitter.split_documents(documents)
len(docs)

838

In [13]:
docs[0].page_content

'Llama 2: Open Foundation and Fine-Tuned Chat Models\nHugo Touvron∗ Louis Martin† Kevin Stone†\nPeter Albert Amjad Almahairi Yasmine Babaei Nikolay Bashlykov Soumya Batra\nPrajjwal Bhargava Shruti Bhosale Dan Bikel Lukas Blecher Cristian Canton Ferrer Moya Chen\nGuillem Cucurull David Esiobu Jude Fernandes Jeremy Fu Wenyin Fu Brian Fuller\nCynthia Gao Vedanuj Goswami Naman Goyal Anthony Hartshorn Saghar Hosseini Rui Hou\nHakan Inan Marcin Kardas Viktor Kerkez Madian Khabsa Isabel Kloumann Artem Korenev'

In [22]:
from langchain.vectorstores import FAISS

vectorstore =  FAISS.from_documents(
    docs,embedding_model)

In [ ]:
#1. in-memory --> (FAISS, Chroma, etc.)
#2. on-disc (FAISS, Chroma, etc.)
#3. cloud storage (pinecone, weaviate, milvus, mongodbvectorsearch, astradb etc.)

In [25]:
relevant_document = vectorstore.similarity_search("what is the llama 2 llm model?")

In [ ]:
# this is retrieval process from vector DB.
relevant_document[0].page_content

'Llama 2\n7B 0.28 0.25 0.29 0.50 0.36 0.37 0.21 0.34 0.32 0.50 0.28 0.19 0.26 0.32 0.44 0.51 0.30 0.2513B 0.24 0.25 0.35 0.50 0.41 0.36 0.24 0.39 0.35 0.48 0.31 0.18 0.27 0.34 0.46 0.66 0.35 0.2834B 0.27 0.24 0.33 0.56 0.41 0.36 0.26 0.32 0.36 0.53 0.33 0.07 0.26 0.30 0.45 0.56 0.26 0.3570B 0.31 0.29 0.35 0.51 0.41 0.45 0.27 0.34 0.40 0.52 0.36 0.12 0.28 0.31 0.45 0.65 0.33 0.20\nFine-tuned'

In [27]:
relevant_document[1].page_content

'Llama 1\n7B 0.27 0.26 0.34 0.54 0.36 0.39 0.26 0.28 0.33 0.45 0.33 0.17 0.24 0.31 0.44 0.57 0.39 0.3513B 0.24 0.24 0.31 0.52 0.37 0.37 0.23 0.28 0.31 0.50 0.27 0.10 0.24 0.27 0.41 0.55 0.34 0.2533B 0.23 0.26 0.34 0.50 0.36 0.35 0.24 0.33 0.34 0.49 0.31 0.12 0.23 0.30 0.41 0.60 0.28 0.2765B 0.25 0.26 0.34 0.46 0.36 0.40 0.25 0.32 0.32 0.48 0.31 0.11 0.25 0.30 0.43 0.60 0.39 0.34\nLlama 2'

In [28]:
relevant_document[2].page_content

'English, and has not covered, nor could it cover all scenarios. For these reasons, as with all LLMs,\nLlama 2’s potential outputs cannot be predicted in advance, and the model may in some instances\nproduce inaccurate or objectionable responses to user prompts. Therefore, before deploying any\napplications ofLlama 2, developers should perform safety testing and tuning tailored to their\nspecific applications of the model. Please see the Responsible Use Guide available available at'

In [29]:
relevant_document[3].page_content

'more recent, up to July 2023.\nEvaluation Results\nSee evaluations for pretraining (Section 2); fine-tuning (Section 3); and safety (Section 4).\nEthical Considerations and Limitations(Section 5.2)\nLlama 2is a new technology that carries risks with use. Testing conducted to date has been in\nEnglish, and has not covered, nor could it cover all scenarios. For these reasons, as with all LLMs,\nLlama 2’s potential outputs cannot be predicted in advance, and the model may in some instances'

In [30]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10}) #can be used to create a retriever chain.

retriever.invoke("llama2 finetuning benchmark experiments.")


[Document(id='6ab877c7-51d1-4319-88a7-6df2c35828e1', metadata={'producer': 'pdfTeX-1.40.25', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-07-20T00:30:36+00:00', 'author': '', 'keywords': '', 'moddate': '2023-07-20T00:30:36+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.25 (TeX Live 2023) kpathsea version 6.3.5', 'subject': '', 'title': '', 'trapped': '/False', 'source': 'd:\\LLMOPS\\DOCUMENT_PORTAL\\notebook\\data\\sample.pdf', 'total_pages': 77, 'page': 48, 'page_label': '49'}, page_content='Llama 1\n7B 76.5 79.8 48.9 76.1 70.1 72.8 47.6 57.2 33.6 35.1\n13B 78.1 80.1 50.4 79.2 73.0 74.8 52.7 56.4 62.0 46.9\n33B 83.1 82.3 50.4 82.8 76.0 80.0 57.8 58.6 72.5 57.8\n65B 85.3 82.8 52.3 84.2 77.0 78.9 56.0 60.2 74.0 63.4\nLlama 2\n7B 77.4 78.8 48.3 77.2 69.2 75.2 45.9 58.6 57.8 45.3\n13B 81.7 80.5 50.3 80.7 72.8 77.3 49.4 57.0 67.3 54.8\n34B 83.7 81.9 50.9 83.3 76.7 79.4 54.5 58.2 74.3 62.6\n70B 85.0 82.8 50.7 85.3 80.2 80.2 57.4 60.2 78.5 68.9\nTable 20:

# Question: user question
# Context: based on the question retrieving the info from the vector database

In [31]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [32]:
from langchain.prompts import PromptTemplate

In [33]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [35]:
# this prompt will go to the LLM model for inference.
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below. \n        If the context does not contain sufficient information, respond with: \n        "I do not have enough information about this."\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:')

In [36]:
from langchain_core.output_parsers import StrOutputParser

In [37]:
parser=StrOutputParser()

In [38]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [40]:

from langchain_core.runnables import RunnablePassthrough

In [42]:
# study LCEL concept

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [43]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

'<think>\nOkay, let\'s try to figure out the answer to the question about the Llama 2 fine-tuning benchmark experiments based on the provided context. First, I need to understand what the user is asking. They want information about the fine-tuning experiments related to Llama 2. \n\nLooking at the context, there are several tables mentioned. The main sections are Table 20, Table 3, and some other data points. The user is specifically interested in the fine-tuning benchmarks. \n\nIn the context, there\'s a mention of "Fine-tuned" models in the tables. For example, under the "Fine-tuned" section, there are entries like "ChatGPT", "MPT-instruct 7B", "Falcon-instruct 7B", and "Llama 2-Chat" with different parameters (7B, 13B, 34B, 70B). These numbers likely correspond to the model sizes. \n\nThe tables have numerical data, but the exact meaning isn\'t clear from the context. However, some sections mention performance metrics. For example, in Table 21, there are code generation results on H

In [44]:
rag_chain.invoke("can you tell me Scaling trends for the reward model?")

'<think>\nOkay, let\'s tackle this question about scaling trends for the reward model. The user wants to know what the scaling trends are, so I need to find relevant information in the provided context.\n\nFirst, I\'ll scan through the context to find mentions of scaling. The first paragraph mentions "Scaling Trends. We study the scaling trends in terms of data and model size for the reward model, fine-tuning different model sizes on an increasing amount of the reward model data collected each week..." So that\'s a key point. They looked at how model size and data volume affect performance.\n\nThe next part says "Figure6reportsthesetrends,showingtheexpectedresultthatlarger models obtain higher performance for a similar volume of data." So larger models perform better with the same data amount. Also, "More importantly, the scaling performance in ranking loss." Hmm, maybe the ranking loss improves with scaling?\n\nThen there\'s mention of a margin term leading to a binary split in reward

In [45]:
rag_chain.invoke("the largest country in ASIA?")

'<think>\nOkay, the user is asking for the largest country in Asia. Let me check the context provided.\n\nLooking at the context, under the "Race and Ethnicity" section, there are mentions of Indian, Chinese, Korean, Mexican, and Indigenous. The percentages are 16.5% for Indian and 16.3% for Chinese. But wait, the question is about the largest country in Asia by area, not population. The context here seems to be talking about demographic descriptors in documents, not geographical facts. The terms like "Indian" and "Chinese" here refer to nationality or ethnicity mentions in the data, not the countries themselves. \n\nThe user\'s question is a straightforward geography question. The context provided doesn\'t actually discuss the population or area of countries in Asia. It\'s analyzing how often certain demographic terms are mentioned in documents. For example, "Indian" is mentioned 16.5% of the time, but that\'s about references in texts, not the population size of India. So the context

# One Small task
Take 10 pdfs keep it in same directory and create RAG on top of it.

26th July:


In next class will discuss about the(will start with the modular coding)

exception module

logger module

doc analyser

doc compare

utils and config

2 class

2 more class api and other module

2 more class for deployment